In [2]:
import tarfile
import numpy as np 
import os
import scipy.io as sio
#import rarfile
import torch
import matplotlib.pyplot as plt
#from hmmlearn import hmm
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import string
import numpy as np
import pandas as pd
import pickle as pkl
from collections import Counter
import random

In [3]:
cwd = os.getcwd()
path = '/'.join(cwd.split('\\'))+'/'

In [2]:
import random
random.seed(0)
test_idx = random.sample(range(1,101), 20)
idx = list(set(range(1,101)) - set(test_idx))

In [3]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(idx)

5

In [4]:
train_idx = []
val_idx = []
for train_index, val_index in kf.split(idx):
    train = np.array(idx)[train_index]
    val = np.array(idx)[val_index]
    train_idx.append(train)
    val_idx.append(val)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader, TensorDataset
class MyDataset(Dataset):
    def __init__(self, data_list, target_list):
        assert len(data_list) == len(target_list)
        self.data_list = data_list
        self.target_list = target_list
        
    def __getitem__(self, index):
        dat = self.data_list[index]
        label = self.target_list[index]
        
        return [dat, label]
    
    def __len__(self):
        return len(self.data_list)

In [7]:
def newsgroup_collate_func(batch):
    data_list = []
    label_list = []
    for datum in batch:
        label_list.append(datum[1])
        data_list.append(datum[0])
        
    x = torch.from_numpy(np.array(data_list)).float().to(device)
    y = torch.LongTensor(np.array(label_list)).to(device)
    return x, y

In [8]:
class RNN(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(RNN, self).__init__()
        
        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.rnn = nn.GRU(1, hidden_size, num_layers, batch_first=True)
        
        self.linear1 = nn.Linear(hidden_size, 3)

  
    def init_hidden(self, batch_size):
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size).float()

        return hidden

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        self.hidden = self.init_hidden(batch_size).to(device)

        _, self.hidden = self.rnn(x, self.hidden)
        final_hidden = self.hidden.view(1, 1, batch_size, self.hidden_size).contiguous().view(batch_size, -1)
        out = self.linear1(final_hidden)
        out = F.log_softmax(out, dim=1)
        return out

In [9]:
def do_train(model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0
    for data, labels in dataloader:
        optimizer.zero_grad()
        y_hat = model(data)
        loss = criterion(y_hat, labels.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * len(data) / len(dataloader.dataset)
    return train_loss

In [10]:
def do_eval(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    y_hat_ls = []
    y_ls = []
    with torch.no_grad():
        for data, labels in dataloader:
            y_hat = model(data)
            loss = criterion(y_hat, labels.long())
            y_hat_ls.append(y_hat)
            y_ls.append(labels)
            val_loss += loss.item() * len(data) / len(dataloader.dataset)
            
    return val_loss, torch.cat(y_hat_ls, dim=0), torch.cat(y_ls, dim=0)
def acc(model, dataloader, criterion):
    val_loss, pred,true = do_eval(
    model = model,
    dataloader = dataloader,
    criterion = criterion
    )
    return val_loss, (torch.exp(pred).max(1)[1] == true).float().mean().item()

In [14]:
val_acc_t = []
for k in range(1,5):
    print("k = {}".format(k+1))
    dat_health = np.zeros([64*2*50, 81])
    for i in range(64):
        t = str(train_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/Z_Healthy/c_Z'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_health[i*50:(i+1)*50,:]=a
    for i in range(64):
        t = str(train_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/O_Healthy/c_O'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_health[(i+64)*50:(i+65)*50,:]=a
    dat_inter = np.zeros([64*2*50, 81])
    for i in range(64):
        t = str(train_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/F_Interictal/c_F'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_inter[i*50:(i+1)*50,:]=a
    for i in range(64):
        t = str(train_idx[k][i]).zfill(3)
        a = np.loadtxt(path + 't/Data/N_Interictal/c_N'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_inter[(i+64)*50:(i+65)*50,:]=a
    dat_ict = np.zeros([64*50, 81])
    for i in range(64):
        t = str(train_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/S_Ictal/c_S'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_ict[i*50:(i+1)*50,:]=a
    dat_health_t = np.zeros([32*50, 81])
    for i in range(16):
        t = str(val_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/Z_Healthy/c_Z'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_health_t[i*50:(i+1)*50,:]=a
    for i in range(16):
        t = str(val_idx[k][i]).zfill(3)
        a = np.loadtxt(path +'/Data/O_Healthy/c_O'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_health_t[(i+16)*50:(i+17)*50,:]=a
    dat_inter_t = np.zeros([32*50, 81])
    for i in range(16):
        t = str(val_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/F_Interictal/c_F'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_inter_t[i*50:(i+1)*50,:]=a
    for i in range(16):
        t = str(val_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/N_Interictal/c_N'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_inter_t[(i+16)*50:(i+17)*50,:]=a
    dat_ict_t = np.zeros([16*50, 81])
    for i in range(16):
        t = str(val_idx[k][i]).zfill(3)
        a = np.loadtxt(path + '/Data/S_Ictal/c_S'+t+'.txt')
        a = a[:50*81].reshape([50, 81])
        dat_ict_t[i*50:(i+1)*50,:]=a
    
    dat_health = np.expand_dims(dat_health, 2)
    dat_inter = np.expand_dims(dat_inter, 2)
    dat_ict = np.expand_dims(dat_ict, 2)
    dat_health_t = np.expand_dims(dat_health_t, 2)
    dat_inter_t = np.expand_dims(dat_inter_t, 2)
    dat_ict_t = np.expand_dims(dat_ict_t, 2)
    
    dat_agg = np.concatenate([dat_health, dat_inter, dat_ict])
    dat_lab = np.concatenate([[0]*128*50, [1]*128*50, [2]*64*50])
    dat_agg_t = np.concatenate([dat_health_t, dat_inter_t, dat_ict_t])
    dat_lab_t = np.concatenate([[0]*32*50, [1]*32*50, [2]*16*50])
    
    BATCH_SIZE = 64
    train_dataset = MyDataset(dat_agg, dat_lab)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                batch_size=BATCH_SIZE,
                                collate_fn=newsgroup_collate_func,
                                shuffle=True)
    val_dataset = MyDataset(dat_agg_t, dat_lab_t)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                batch_size=BATCH_SIZE,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)
    
    hidden_size = 32
    num_layers = 1
    learning_rate = 0.001
    model = RNN(hidden_size, num_layers).to(device)
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    num_epochs = 100
    best_acc = 0
    for epoch in range(num_epochs):
    #scheduler.step()
        loss = do_train(
            model=model, 
            criterion=criterion,
            dataloader=train_loader,
            optimizer=optimizer,
        )
        val_loss, val_acc = acc(model, val_loader, criterion)
        if(val_acc > best_acc):
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pkl")
        train_loss, train_acc = acc(model,train_loader, criterion)
        print('Epoch: [{}/{}], Train Loss: {}, Val Loss: {}, Val Acc: {}'.format( 
                           epoch+1, num_epochs, train_loss, val_loss, val_acc))
        
    model.load_state_dict(torch.load("best_model.pkl"))
    t_dataset = MyDataset(dat_agg_t, dat_lab_t)
    t_loader = torch.utils.data.DataLoader(dataset=t_dataset,
                                batch_size=50,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)
    pred_lab = []
    for x, y in t_loader:
        y_hat = model(x)
        pred = torch.exp(y_hat).max(1)[1].numpy()
        pred_lab.append(np.argmax([len(pred[pred==0]), len(pred[pred==1]), len(pred[pred==2])]))
    true_lab = np.concatenate([[0]*32,[1]*32,[2]*16])
    val_acc_t.append(len(true_lab[np.array(pred_lab)==true_lab])/80)

k = 2
Epoch: [1/100], Train Loss: 1.0329049046039591, Val Loss: 1.0364032821655271, Val Acc: 0.5009999871253967
Epoch: [2/100], Train Loss: 0.8386963739395141, Val Loss: 0.8443837757110593, Val Acc: 0.6495000123977661
Epoch: [3/100], Train Loss: 0.7969148950576782, Val Loss: 0.8089122128486631, Val Acc: 0.6622499823570251
Epoch: [4/100], Train Loss: 0.7661393222808839, Val Loss: 0.7771106975078583, Val Acc: 0.6697499752044678
Epoch: [5/100], Train Loss: 0.7325460927486417, Val Loss: 0.7544301345348357, Val Acc: 0.6837499737739563
Epoch: [6/100], Train Loss: 0.7085424705743781, Val Loss: 0.7189963569641111, Val Acc: 0.7067499756813049
Epoch: [7/100], Train Loss: 0.6848348517417907, Val Loss: 0.7058711342811586, Val Acc: 0.7110000252723694
Epoch: [8/100], Train Loss: 0.6538910772800444, Val Loss: 0.671654185771942, Val Acc: 0.7287499904632568
Epoch: [9/100], Train Loss: 0.6346396754980085, Val Loss: 0.6536511774063112, Val Acc: 0.7379999756813049
Epoch: [10/100], Train Loss: 0.6236554404

In [20]:
print("validation accuracy: {}".format(np.mean(val_acc_t)))

validation accuracy: 0.9574999999999999


In [15]:
dat_health_test = np.zeros([40*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt(path + '/Data/Z_Healthy/c_Z'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_test[i*50:(i+1)*50,:]=a
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt(path + '/Data/O_Healthy/c_O'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_test[(i+20)*50:(i+21)*50,:]=a
dat_inter_test = np.zeros([40*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt(path + '/Data/F_Interictal/c_F'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_test[i*50:(i+1)*50,:]=a
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt(path + '/Data/N_Interictal/c_N'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_test[(i+20)*50:(i+21)*50,:]=a
dat_ict_test = np.zeros([20*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt(path + '/Data/S_Ictal/c_S'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_ict_test[i*50:(i+1)*50,:]=a

In [16]:
dat_health_test = np.expand_dims(dat_health_test, 2)
dat_inter_test = np.expand_dims(dat_inter_test, 2)
dat_ict_test = np.expand_dims(dat_ict_test, 2)

In [17]:
dat_agg_test = np.concatenate([dat_health_test, dat_inter_test, dat_ict_test])
dat_lab_test = np.concatenate([[0]*40*50, [1]*40*50, [2]*20*50])
test_dataset = MyDataset(dat_agg_test, dat_lab_test)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                batch_size=50,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)

In [18]:
pred_lab = []
for x, y in test_loader:
    y_hat = model(x)
    pred = torch.exp(y_hat).max(1)[1].numpy()
    pred_lab.append(np.argmax([len(pred[pred==0]), len(pred[pred==1]), len(pred[pred==2])]))
true_lab = np.concatenate([[0]*40,[1]*40,[2]*20])
print("test accuracy: {}".format(len(true_lab[np.array(pred_lab)==true_lab])/100))

test accuracy: 0.97
